# import moudles into our code

In [3]:
from skimage.feature import blob_log
from skimage.color import rgb2gray
import numpy as np
import cv2


find coreds in picter and save to some file

In [4]:
def find_coordinates(img_path, output_path=None):
    #read the inage data and convert it to gray insted of red-blue-green
    image = cv2.imread(img_path)
    gray = rgb2gray(image)

    #detecet ares which have spiks in the brightness
    blobs_log = blob_log(gray, max_sigma=30, num_sigma=10, threshold=.1)

    #for each blob calc the y,x,r and brightness and store in the list
    coordinates = []
    for i, blob in enumerate(blobs_log):
        y, x, r = blob
        brightness = np.mean(gray[int(y - r):int(y + r), int(x - r):int(x + r)])
        coordinates.append((x, y, r, brightness))
        
    #save to output file
    if output_path is not None:
        with open(output_path, 'w') as f:
            for result in coordinates:
                f.write(f"{result[0]},{result[1]},{result[2]},{result[3]}\n")
    return coordinates


function to calc the distence betweeen 2 coreds

In [5]:
def distance(point1, point2):
    #the formla of distance between 2 points
    return np.sqrt((point1[0] - point2[0]) ** 2 + (point1[1] - point2[1]) ** 2)

using the 2 function above to find the best natchs and return it

In [6]:
def match_stars(img1_path, img2_path, output_path=None):
    #find the 2 list of cords
    stars1 = find_coordinates(img1_path)
    stars2 = find_coordinates(img2_path)
    matching_pairs = []
    max_matching_score = 1
    
    #for each cord in image1
    for star1 in stars1:
        best_matching_score = max_matching_score
        matching_star = None

        #check against each start in image2 
        for star2 in stars2:
            dist = distance(star1, star2)

            if dist < 50:
                matching_score = 1 - (dist / 50)
                
                #if it it the best match - update the matching_star value
                if matching_score < best_matching_score:
                    best_matching_score = matching_score
                    matching_star = star2

        #if we found a match - we will remove from the image2 list and add to the matching starts
        if matching_star is not None:
            matching_pairs.append((star1, matching_star))
            stars2.remove(matching_star)

    #save to output file
    if output_path is not None:
        with open(output_path, "w") as file:
            for pair in matching_pairs:
                file.write(str(pair) + "\n")

    return matching_pairs

In [7]:
print(find_coordinates("right_image.jpg", "stam.txt"))

[(134.0, 123.0, 4.222222222222222, 0.6967026821592834), (324.0, 366.0, 4.222222222222222, 0.7774177874606633), (346.0, 450.0, 4.222222222222222, 0.7528303316388284), (229.0, 65.0, 4.222222222222222, 0.599986952311789), (454.0, 531.0, 4.222222222222222, 0.7180392253691601), (337.0, 188.0, 4.222222222222222, 0.623711188574195)]


In [8]:
print(find_coordinates("left_image.jpg", "stam2.txt"))

[(523.0, 600.0, 1.0, 0.7403168627450981), (425.0, 516.0, 1.0, 0.5467478431372549), (552.0, 34.0, 1.0, 0.31729558823529413), (448.0, 561.0, 1.0, 0.5258901960784312), (243.0, 196.0, 1.0, 0.42168941176470587), (58.0, 609.0, 1.0, 0.6095485294117646), (239.0, 657.0, 1.0, 0.5968703921568628), (409.0, 423.0, 1.0, 0.4973936274509805), (351.0, 74.0, 1.0, 0.45882352941176463), (343.0, 377.0, 1.0, 0.4680525490196079), (393.0, 466.0, 1.0, 0.4847192156862746), (304.0, 413.0, 1.0, 0.585206568627451), (447.0, 309.0, 1.0, 0.3696636274509804), (408.0, 222.0, 1.0, 0.3939075490196078), (477.0, 22.0, 1.0, 0.22898960784313727), (254.0, 191.0, 1.0, 0.3419986274509804), (275.0, 258.0, 1.0, 0.44445245098039216), (517.0, 206.0, 1.0, 0.3054601960784314)]


In [9]:
print(match_stars("right_image.jpg", "left_image.jpg", "stam3.txt"))

[((324.0, 366.0, 4.222222222222222, 0.7774177874606633), (343.0, 377.0, 1.0, 0.4680525490196079)), ((346.0, 450.0, 4.222222222222222, 0.7528303316388284), (393.0, 466.0, 1.0, 0.4847192156862746)), ((454.0, 531.0, 4.222222222222222, 0.7180392253691601), (425.0, 516.0, 1.0, 0.5467478431372549))]
